In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json

headlineList = []
authorList = []
dateList = []
categoryList = []


def getNews(html):
    soup = BeautifulSoup(html,'lxml')
    divs = soup.find_all('div', {'class': "news-card-title news-right-box"})
    
    #Extracting News Headlines
    newsHeadlines = [div.find('span',{'itemprop':'headline'}).string for div in divs]
    headlineList.extend(newsHeadlines)
    
    #Extracting author
    authors = [div.find('span',class_ = "author").string for div in divs]
    authorList.extend(authors)
    
    #Extracting news date
    newsDates = [div.find('span',clas = "date").string for div in divs]
    dateList.extend(newsDates)
    
    #Extracting News Category
    categories = [newsCategory for div in divs]
    categoryList.extend(categories)
    
    
def getMoreNewsId(response_text):
    soup = BeautifulSoup(response_text, 'lxml')
    patrn = re.compile('var min_news_id\s+=\s+"(.*?)"')
    new_id_scr = soup.find("script", text=re.compile("var\s+min_news_id"))
    news_id = patrn.search(new_id_scr.text).group(1)
    return news_id


baseURL = 'https://inshorts.com/en/read/'
newsCategoryList = ['national','business','sports','world','politics','technology','startup','entertainment','miscellaneous','hatke','science','automobile']


for newsCategory in newsCategoryList:
    url = baseURL + newsCategory
    r = requests.get(url)
    r.status_code
    html = r.content
  
    getNews(html)
    
    #Min News Id
    moreNewsUrl = "https://inshorts.com/en/ajax/more_news"
    news_id = getMoreNewsId(html)
    
    i = 0
    while i<2:        
        response = requests.post(moreNewsUrl, data={"category": '', "news_offset": news_id})
        if response.status_code != 200:
            print(response.status_code)
            break
                
        response_json = json.loads(response.text)
        getNews(response_json["html"])
        news_id = response_json["min_news_id"]
        i = i + 1
        
df = pd.concat([pd.Series(categoryList,name = "Category"),pd.Series(headlineList,name = "Headline"), pd.Series(authorList,name = "Author"), pd.Series(dateList, name = "Date")],axis = 1)
# df.to_excel("FinalNews.xlsx",index=False)
df

,Category,Headline,Author,Date
0,national,"Tamil Nadu reports 25 new COVID-19 cases, tota...",Kiran Khatri,"16 Apr 2020,Thursday"
1,national,"Gujarat reports total 929 coronavirus cases, d...",Kiran Khatri,"16 Apr 2020,Thursday"
2,national,No new confirmed coronavirus case in Himachal ...,Anmol Sharma,"16 Apr 2020,Thursday"
3,national,Total number of coronavirus cases in Maharasht...,Nandini Sinha,"16 Apr 2020,Thursday"
4,national,55 new coronavirus cases reported in Rajasthan...,Kiran Khatri,"16 Apr 2020,Thursday"
...,...,...,...,...
912,automobile,Coronavirus cases rise to 14 in NBA as 2 LA La...,Ankur Taliyan,"20 Mar 2020,Friday"
913,automobile,Can't ask pilgrims not to attend Ram Navami me...,Atul Mishra,"20 Mar 2020,Friday"
914,automobile,Apple limits online iPhone purchases to 2 per ...,Aishwarya,"20 Mar 2020,Friday"
915,automobile,Victory of truth: Scindia after CM Kamal Nath ...,Kirti Joshi,"20 Mar 2020,Friday"
